In [1]:
import os, sys
import json
import math
import numpy as np
import cv2 
from time import time

sys.path.append('/home/loitg/workspace/backbone/src/')
from utils import Coord, Line
from loi import createStandardGrid, multiple_transform, centralize, mls, alignGuidelines, gls2arr,\
drawField, arr1gls, arr2gls, GridMap, reconstructFromField
from datagen import syn_augment_matrix

In [2]:
import sqlite3
ROOT_DIR = '/home/loitg/workspace/backbone/'

db = sqlite3.connect(os.path.join(ROOT_DIR, 'flaskr.sqlite'), detect_types=sqlite3.PARSE_DECLTYPES)
c = db.cursor()
rs = c.execute("SELECT name FROM sqlite_master WHERE type='table' AND name=?", ['images']);

rs = c.execute("select * from images")

allrows = rs.fetchall()
imageName_all = [row[0] for row in allrows if row[1] != 'None']
imageStatus_all = [row[1] for row in allrows if row[1] != 'None']
guidelines_all = [json.loads(row[2]) for row in allrows if row[1] != 'None']

def cvtGL2Coords(guidelines):
    return [[Coord(a["x"], a["y"]) for a in guideline] for guideline in guidelines]

guidelines_all = [cvtGL2Coords(guidelines) for guidelines in guidelines_all]

N = len(imageName_all)

In [3]:
pbin = '/home/loitg/ssd/data/temp3/protobuf_unify_old'
imagesin = '/home/loitg/ssd/data/temp3/images_unify_old'
# hinhphang1 = '/home/loitg/workspace/dewarp/hinhphang1'
imagestemp = '/home/loitg/workspace/backbone/imagestemp/'

sys.path.append('/home/loitg/workspace/test_google/')

from pbprocess import get_document_bounds, FeatureType, readPb, draw_bounds, bound2points

def inputsFromFile(fn, image_path, pb_path):
    image = cv2.imread(os.path.join(image_path, fn)) # hinh cong tu nhien
    pb_response = readPb(os.path.join(pb_path, fn.replace('jpg', 'pb') ))
    word_bounds = get_document_bounds(pb_response, FeatureType.WORD)
    boxes = [bound2points(bound) for bound in word_bounds]
    mask_word = np.zeros(image.shape[:2], dtype=np.uint8)
    return image, mask_word

In [7]:
#     # Augmentatino
#     t, r = syn_augment_matrix(image, None, None)
#     xx0, yy0 = t.transform_points(xx0, yy0)
#     xx1, yy1 = t.transform_points(xx1, yy1)
#     new_image = t.transform_image(image)
#     new_mask = t.transform_image(mask_word)
def process(image, mask_word, gls):
    if len(gls) == 0:
        return image, mask_word, image, np.zeros(shape=(image.shape[0], image.shape[1], 2))
    
    tt = time()
    
    a_gls = alignGuidelines(gls)
    
    print('%f: alignGuidelines' % (time() - tt))
    tt = time()
    
    control_points = gls2arr(gls)
    dst_points = gls2arr(a_gls)

    print('%f: gls2arr' % (time() - tt))
    tt = time()
    
    xx0, yy0 = createStandardGrid(image.shape)
    grid_shape = xx0.shape
    xx0 = xx0.reshape(-1)
    yy0 = yy0.reshape(-1)

    standargrid = np.stack([yy0,xx0], axis=1)

    print('%f: create standardgrid' % (time() - tt))
    tt = time()
    
    newgrid = mls(control_points, dst_points, standargrid)
    xx1 = newgrid[:,1]
    yy1 = newgrid[:,0]
    xx1, yy1 = centralize(xx1, yy1)
    
    print('%f: mls' % (time() - tt))
    tt = time()
    
    # Augmentatino
    t, r = syn_augment_matrix(image, None, None)
    xx0, yy0 = t.transform_points(xx0, yy0)
    xx1, yy1 = t.transform_points(xx1, yy1)
    augmented_image = t.transform_image(image)
    augmented_mask = t.transform_image(mask_word)

    print('%f: augment' % (time() - tt))
    tt = time()
    
    mapping = GridMap(np.stack([yy0, xx0], axis=1), np.stack([yy1, xx1], axis=1))
    print('%f: f and f_inverse' % (time() - tt))
    tt = time()
    flatten = mapping.transformArrayScalar_bilinear(augmented_image)
    print('%f: transform_bilinear' % (time() - tt))
    tt = time()
    deviation_field = mapping.deviationFromShape(image.shape)
    print('%f: deviaiton_field' % (time() - tt))
    tt = time()
    return augmented_image, augmented_mask, flatten, deviation_field

In [8]:
for i in range(N):
    fn = imageName_all[i]
#     if fn != '650000000002571.85b265c4-35fc-4fe0-a240-d9672390ea54.636615750414373359.jpg':
#         continue
    print(fn)
    image, mask_word = inputsFromFile(fn, imagesin, pbin)
    gls = guidelines_all[i]
    aug_image, aug_mask, flatten, deviation_field = process(image, mask_word, gls)
    
    tt = time()
    reconstructed = reconstructFromField(aug_image, deviation_field)
    
    print('%f: reconstruct' % (time() - tt))
    tt = time()
    
    cv2.imwrite(os.path.join(imagestemp, '%s_folded.jpg' % (fn)), aug_image)
    cv2.imwrite(os.path.join(imagestemp, '%s_recons.jpg' % (fn)), reconstructed) 

650000000000030.9f127175-977c-4579-8010-b519185d3f52.636613972057668675.jpg
0.013140: alignGuidelines
0.000118: gls2arr
0.002727: create standardgrid
0.089079: mls
8.389470943508803
0.055772: augment
0.315959: f and f_inverse
0.996393: transform_bilinear
0.340807: deviaiton_field
0.112706: reconstruct
650000000000051.8e89d6c1-99a3-40d2-ac21-db1d2544b43e.636615671966775367.jpg
0.000482: alignGuidelines
0.000091: gls2arr
0.000970: create standardgrid
0.138383: mls
0.7928846020093215
0.023256: augment
0.254734: f and f_inverse
0.967744: transform_bilinear
0.369143: deviaiton_field
0.095757: reconstruct
650000000000055.a12cff97-ff45-4f8e-8639-5374b665a426.636610805786953293.jpg
0.001371: alignGuidelines
0.000124: gls2arr
0.000797: create standardgrid
0.184060: mls
4.7020831335622235
0.020998: augment
0.344496: f and f_inverse


KeyboardInterrupt: 

In [ ]:
####### cong tu nhien =>align=> 
control_points = gls2arr(gls)
dst_points = gls2arr(a_gls)

xx0, yy0 = createStandardGrid(image.shape)
grid_shape = xx0.shape
xx0 = xx0.reshape(-1)
yy0 = yy0.reshape(-1)

standargrid = np.stack([yy0,xx0], axis=1)

newgrid = mls(control_points, dst_points, standargrid)
xx1 = newgrid[:,1]
yy1 = newgrid[:,0]
xx1, yy1 = centralize(xx1, yy1)

mapping = GridMap(np.stack([yy0, xx0], axis=1), np.stack([yy1, xx1], axis=1))

#########
new_image = mapping.transformArrayScalar_bilinear(image)
cv2.imwrite(os.path.join(imagestemp, '%s.jpg' % (fn)), new_image)

deviation_field = mapping.deviationFromShape(image.shape)
# deviation_field = drawField(image, deviation_field)
# cv2.imwrite(os.path.join(imagestemp, '%s_deviation_field.jpg' % (fn)), deviation_field)

# new_image, new_mask, new_vmap, F = mapping.transformAll(image, mask_word, direction_map)

# cv2.imwrite(os.path.join(imagestemp, '%s_labeler_mask.jpg' % (fn)), new_mask*255)
# new_vmap = drawField(new_image, new_vmap)
# cv2.imwrite(os.path.join(imagestemp, '%s_labeler_vmap.jpg' % (fn)), new_vmap)
# F = drawField(image, F)
# cv2.imwrite(os.path.join(imagestemp, '%s_labeler_F.jpg' % (fn)), F)